![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

# Travailler avec des données ouvertes Partie 3 - La loterie

## Introduction
Ce cahier démontre certaines compétences de manipulation de données plus complexes. Cette fois, nous utiliserons un ensemble de données des numéros gagnants historiques de la loterie.

## L'ensemble de données

Nous utiliserons les données historiques Lotto 649 hébergées sur le site Web Lotto 649 [à ce lien](http://www.lotto649stats.com/recent_winning.html). Si vous choisissez de suivre ce lien, vous vous retrouverez avec une table qui ressemble à celle ci-dessous

![winning lottery numbers](images/lotto.png)

Ce qui est fantastique, il y a toutes les données là. Cependant, la question demeure: comment allons-nous extraire ces données du site Web dans un format qui fonctionnera bien pour nous? Une autre prise, qui n'est pas évidente à partir de la capture d'écran, est qu'il s'agit d'un tableau incomplète. Plusieurs autres tableaux des années précédentes sont disponibles. Comme il s'agit de données ouvertes, nous sommes libres de parcourir ce site Web (manuellement ou avec un ordinateur) et de copier ces données nous-mêmes. Cependant, cela pourrait prendre beaucoup plus de temps que cela en vaut la peine, alors nous devrions hésiter à commencer notre analyse à partir d'ici.

Il s'avère que quelqu'un d'autre a déjà pris la peine de saisir les données depuis une trentaine d'années et de les intégrer dans un format facile à utiliser. Ces données sont disponibles [at this link](https://www.kaggle.com/datascienceai/lottery-dataset). Malheureusement, ceci est un exemple de "données semi-ouvertes". Vous êtes libre de le télécharger sans restriction, mais vous devez vous inscrire pour accéder au site Web. Heureusement, nous avons déjà fait ces efforts pour vous et nous chargeons simplement une copie locale enregistrée séparément. Vous trouverez ci-dessous quelques lignes de code qui devraient sembler familières du cahier précédent.

In [ ]:
# Import our data manipulation library
import pandas as pd 
import matplotlib.pyplot as plt
# So any plots we want will appear in the notebook
%matplotlib inline

In [ ]:
# Here we're loading the data set, here the 'pd' prefix tells us that 'read_csv' is a method
# coming from the pandas library
lottery = pd.read_csv("data/649.csv")

# This is to show us how many rows in our table we will have, 'len' stands for length
print('Total number of lotteries played:', len(lottery))

# This is to just look at the first five rows of the data set and keep the 
# notebook a little cleaner. 
lottery.head()

Le tableau ci-dessus est constitué des cinq premières lignes de notre tableau qui contient des entrées pour 3665 loteries distinctes. C'est beaucoup de prix! Une première question naturelle à poser à propos de ces données est "est-ce que certains nombres sont plus populaires que d’autres?". S'il y a un préférence pour certains numéros par rapport à d'autres, nous aimerions plutôt jouer ces numéros!

Un bon moyen d’essayer est de compter le nombre de fois que chaque numéro apparaît. Par exemple, combien de 1, 2, 3 ... etc. ont été dessinés tout au long de la vie de cet ensemble de données. Passons en revue les étapes à suivre. Commençons par créer un nouveau tableau de données contenant uniquement les données numériques.

In [ ]:
# First we define a list of the column names (seen in the table above) that we are interested in in order
# to count up their entries. 
cols = ['NUMBER DRAWN 1','NUMBER DRAWN 2','NUMBER DRAWN 3',
        'NUMBER DRAWN 4','NUMBER DRAWN 5','NUMBER DRAWN 6']

# By passing the list we've defined above, we return only those columns. We then assign those columns
# to a new dataframe called 'numbers'
numbers = lottery[cols]

numbers.head()

Maintenant que nous avons isolé les nombres d'intérêt, il suffit de compter toutes les occurrences de chaque nombre. Avant de pouvoir faire cela, nous devons d'abord introduire une autre fonctionnalité Python / pandas: la fonction `apply()`.

---
### Digression: Comprendre Apply ()

Apply est une fonction de python / pandas qui vous permet "d'appliquer" une autre fonction ou transfomation facilement à une grande quantité de données. Dans notre cas, considérez-le comme un outil que nous utilisons pour manipuler toutes les données de notre cadre à la fois. Ci-dessous, nous travaillons sur un exemple de la façon Apply fonctionne en montrant comment nous pouvons l'utiliser pour faire quelque chose de faussement simple: ajoutez-en une à chaque entrée d'une trame de données.

In [ ]:
'''
Here we're creating a new dataframe and filling it with zeroes. 

The first argument is the values to fill (here just zero everywhere) 
index is a list of labels for the row indexes, here just zero through five,
and columns is the column names, again zero through five 
'''

example = pd.DataFrame(0, index=[0,1,2,3,4,5], columns=[0,1,2,3,4,5])
example

In [ ]:
'''
Now we're going through defining a function, a process which we need to introduce. 

1. Here the 'def' keyword can be thought of to mean "define"

2. 'add_one' is the name we've given to the function that we're defining. This choice is 
    arbitrary and we could have named it anything we wanted. However, it's often helpful 
    to name a function in a way that is meaningful to help describe the purpose of the function.
    
3. 'x' in parenthesis is the name we're assigning to the variable that our function will be 
   taking as input 
   
4. output is the name that we're giving to a variable internal to our function to make calculations

5. return is the keyword that tells our function to return that value once it's been called

6. Notice the consistent indentation after 'def', this is how python knows that the lines of code 
   underneath def are a part of the function 


'''

# This function will take a number x as input, and return that value plus one 
def add_one(x):
    output = x + 1
    return output

# Here we're testing it for expected behavior: Does it indeed 
# return the original number plus one? 
add_one(10)

Maintenant que nous avons défini une trame de données "test" et une fonction qui en ajoutera un à n'importe quelle entrée, nous démontrerons ce que la méthode `apply()` fera à une trame de données.

In [ ]:
'''
Notice how we're not supplying any arguments to our add_one function, that is because 
the arguments will be supplied from the contents of the dataframe 'example' itself!
'''
example.apply(add_one)

Remarquez comment nous avons ajouté un à chaque entrée de notre trame de données en une seule ligne de code! Pour nos besoins, `apply()` peut être considéré comme une méthode qui 'applique' une fonction à notre trame de données. La fonctionnalité de `apply()` s'étend bien au-delà. Pour nos besoins, cela devrait être suffisant de comprendre les sections suivantes.


---


### De retour aux affaires

Nous allons maintenant montrer comment utiliser `apply()` pour compter combien de fois chaque nombre est apparu dans la loterie. Nous allons utiliser une fonction de pandas interne, similaire à une fonction Excel telle que `SUM`,` AVERAGE`, `MEAN`, etc. Dans notre cas, nous allons utiliser `pd.value_counts`. Le préfixe `pd` spécifie qu'il s'agit d'une fonction pandas, et `value_counts` est le nom de la fonction que nous utilisons. Ce que `value_counts` fait est elle renvoie un objet de trame de données séparé des nombres de toutes les valeurs uniques. Voyons comment l'utiliser avec apply pour compter les fréquences des numéros de loterie ci-dessous.

In [ ]:
"""
Here the function we're passing to "apply" is pd.value_counts. This is a pandas function
that counts the occurences of unique entries, in this case in the data frame. This is the perfect function
to count how many times each number has been drawn in the lottery!
"""

numbers.apply(pd.value_counts)

Là où dans le tableau ci-dessus, nous pouvons voir combien de fois chaque nombre a été compté et dans quelle position! Cela peut également représenter un bon exemple de cas où des données parfaitement correctes peuvent sembler trompeuses. Pourquoi est-ce que beaucoup de ces valeurs de `NaN` apparaissent à nouveau? Eh bien, la réponse est assez simple. Comme ces valeurs sont triées par ordre croissant dans chaque ligne, nous constatons que certains numéros ne peuvent pas apparaître dans certaines positions. Par exemple, en raison du tri, le nombre 1-5 ne peut _jamais_ être présent dans la 6$^{\text{e}}$ colonne.

Un tableau de données est merveilleux, mais il est beaucoup plus facile de communiquer quelque chose comme un compte de fréquence sous forme d'histogramme. Cependant, nous devons d'abord additionner le nombre de fois où chaque nombre a été compté! Dans Excel, une telle affaire serait assez simple; Tapez simplement `SUM`, faites glisser une sélection de lignes, puis faites glisser cette cellule sur le côté. Facile! Ici, nous allons voir que c'est aussi facile, peut-être même plus facile en utilisant pandas.

In [ ]:
'''
Here we first create a new data frame 'counts' which is the dataframe we created before

We then create a new  "sum_column" in our data frame, and filling it with the sum of each 
row in our data frame. The sum of each row is specified by the 'axis=1' argument.
'''

counts = numbers.apply(pd.value_counts)

# Create a new column in our counts data frame called "sum_column"
counts["sum_column"] = counts.sum(axis=1)

counts

L'action ci-dessus a ajouté une nouvelle colonne qui représente la somme des données contenues dans chaque ligne. Notez que les entrées `NaN` ne sont pas comptées dans le total numérique de notre colonne de somme (plus facile à voir dans la première ou la dernière ligne). Maintenant que nous avons une colonne pratique, visualisons enfin le nombre de comptes que nous avons.

In [ ]:
'''
Here we're simply making a plot of the number frequencies that we counted in the previous step.

'''

counts['sum_column'].plot(kind = 'bar', figsize = (16,8))
plt.xlabel("Number Drawn", size = 16)
plt.ylabel("Counts", size = 16)
plt.title("Number Frequencies of Lotto 649", size = 18)

D'après la figure ci-dessus, il est clair qu'il n'y a pas nécessairement une distribution _uniforme_ des numéros de loterie. Bien sûr, la question demeure; y a-t-il des numéros _plus susceptibles_ d'être tirés à la loterie que d'autres? Si tel était le cas, il serait préférable de choisir ces numéros à la place.

Maintenant, dans le cadre de l'ensemble du cahier, il semble que nous ayons beaucoup travaillé pour créer le graphique ci-dessus. C'est tout le contraire! Dans la cellule ci-dessous, nous avons répété tout le code nécessaire pour charger l'ensemble de données jusqu'à la création de l'histogramme de fréquence du Loto 649. En fait, en ignorant les espaces et les commentaires, seulement dix lignes de code sont nécessaires de passer du chargement des données à la création d'un graphique formaté.

In [ ]:
# Below is all the code required to create the histogram of Lotto 649 number draws 
# Libraries
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

# Data loading/wrangling
lottery = pd.read_csv("data/649.csv")

cols = ['NUMBER DRAWN 1','NUMBER DRAWN 2','NUMBER DRAWN 3',
        'NUMBER DRAWN 4','NUMBER DRAWN 5','NUMBER DRAWN 6']
numbers = lottery[cols]

counts = numbers.apply(pd.value_counts)
counts["sum_column"] = counts.sum(axis=1)

# To create the plot again remove the '#' and trailing space from the four lines
# below. 

# counts['sum_column'].plot(kind = 'bar', figsize = (16,8))
# plt.xlabel("Number Drawn", size = 16)
# plt.ylabel("Counts", size = 16)
# plt.title("Number Frequencies of Lotto 649", size = 18)

## Conclusion

Espérons que ce cahier a démystifié la façon d'intégrer des données ouvertes dans un cahier Jupyter. Nous avons démontré la simplicité relative du chargement des données dans une trame de données facile à utiliser, ensuite d'utiliser cette trame de données pour créer et visualiser directement des statistiques sommaires. Bien entendu, ce n’est que le début de toute analyse. Si vous voulez comprendre des "anomalies non aléatoires", vous devrez approfondir cette trame de données. Peut-être effectuer un test de hasard comme un test de chi carré ou de Kolmogorov-Smirnov. Vous essayerez de trouver des nombres qui semblent être "moins aléatoires" que les autres et les utiliser comme des choix favorables dans vos propres choix de loterie. Certes, il serait surprenant que la loterie soit sujette à des préférences, mais la découverte de ces possibilités est précisément le but de travailler avec des données ouvertes.


---

## Bonus

Vous trouverez ci-dessous une fonction appelée `lot` qui générera des numéros de loterie aléatoires pour vous. Si vous vous sentez aventureux, essayez de créer votre propre ensemble de données (simulé) de numéros de loterie. Vous pouvez utiliser cet ensemble de données synthétiques pour comparer et contraster avec les données réelles. Ou démarrer simplement la cellule plusieurs fois pour générer autant de combinaisons de numéros de loterie potentiels que vous le souhaitez.

In [ ]:
import random
from IPython.display import clear_output
def lot(sort = True):
    # Create a list (the [] brackets) of your firstrandom lottery number
    choice = [float(random.randint(1,49))]
    
    # This is an infinite loop: be careful!
    while True:
        # Try to add a new number to our list provided it isn't already in 
        # our list
        new = float(random.randint(1,49))
        if new not in choice:
            # If it's a number we don't already have, add it to the list
            choice.append(new)
        
        # If we have 6 numbers, we can exit our infinite loop by returning
        # our lottery choices
        if len(choice) == 6:
            if sort:
                return sorted(choice)
            else:
                return choice

# This actually calls our function
lot()



[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)